Importing Libraries

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms, models
import torch.optim as optim
import os
import random
import cv2
import numpy as np
from PIL import Image

Labels

In [ ]:
class_label_real = 0
class_label_attack = 1

Testing Synthetic Images Dataset Path

In [ ]:
data_path_test_synthetic_images = '/folder/test_synthetic_images'

Function for Loading Images

In [ ]:
def load_samples_image(image_path, class_label, transform):
    image = Image.open(image_path).convert('RGB') # It uses PIL (Pillow) library to open the image, convert it to the RGB mode
    sample = (transform(image), class_label) # Apply transformation
    return sample

class ImageDataset(Dataset):
    def __init__(self, data_path, class_label):
        self.data_path = data_path
        self.image_files = [file for file in os.listdir(data_path) if file.endswith(('.png', '.jpg', '.jpeg'))]
        self.class_label = class_label
        self.data_length = len(self.image_files)
        self.transform = transforms.Compose([transforms.Resize((256, 256)),
                                             transforms.ToTensor()])

    def __len__(self):
        return self.data_length

    def __getitem__(self, idx):
        file = self.image_files[idx]
        path = os.path.join(self.data_path, file)
        sample = load_samples_image(path, self.class_label, self.transform)
        return sample

Loading Dataset & Assigning Labels 

In [ ]:
test_dataset_attack_images = ImageDataset(data_path_test_synthetic_images, class_label_attack)

Dataset Concatenation & Loading

In [ ]:
test_loader_attack_images = DataLoader(test_dataset_attack_images, batch_size=64, shuffle=False, pin_memory=True, num_workers=8)

Checking Total No. of Samples for Testing

In [ ]:
# Print dataset sizes
print(f"Test set size: {len(test_dataset_attack_images)}")

Loading Pre-trained Model

In [ ]:
# Load pre-trained MobileNetV2
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(in_features=1280, out_features=2) #default in_features =1280, out_features = 1000
# print(model)

Loading Trained Model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the path to your saved model file
model_path = '/folder/bestmodel_fake.pth'

# Load the saved model
checkpoint = torch.load(model_path)

# Load the model's state dictionary
model.load_state_dict(checkpoint)

Testing

In [ ]:
# Evaluate on the test set
test_correct = 0
test_total = 0

model.eval()
with torch.no_grad():
    
    test_cat_labels = torch.empty(0, dtype=torch.int64, device=device)
    test_predicted_cat_labels = torch.empty(0, dtype=torch.int64, device=device)

    for test_images, test_labels in test_loader_attack_images:
        test_images, test_labels = test_images.to(device), test_labels.to(device)
        test_model_op = model(test_images)
        _, test_predicted = torch.max(test_model_op, 1)
        test_correct += (test_predicted == test_labels).sum().item() 
        test_total += test_labels.size(0)

        test_cat_labels = torch.cat((test_cat_labels, test_labels))
        test_predicted_cat_labels = torch.cat((test_predicted_cat_labels, test_predicted))

    test_accuracy = test_correct / test_total * 100  
    print(f'Test Accuracy: {test_accuracy:.2f}%')

In [ ]:
test_cat_labels_cpu = test_cat_labels.cpu()
test_predicted_cat_labels_cpu = test_predicted_cat_labels.cpu()
print(test_cat_labels_cpu)
print(len(test_cat_labels_cpu))
print(test_predicted_cat_labels_cpu)
print(len(test_predicted_cat_labels_cpu))

Evaluation Metrics

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score

tn, fp, fn, tp = confusion_matrix(test_cat_labels_cpu, test_predicted_cat_labels_cpu, labels=[0,1]).ravel()

print(f'TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}')

acc_score = accuracy_score(test_cat_labels_cpu, test_predicted_cat_labels_cpu)
prec_score = precision_score(test_cat_labels_cpu, test_predicted_cat_labels_cpu)
recall = recall_score(test_cat_labels_cpu, test_predicted_cat_labels_cpu)

Y_I_val =(tp/(tp+fn)) + (tn/(tn+fp)) - 1
sensitivity_val = tp / (tp + fn)
specificity_val = tn / (tn + fp)
f1score_val = 2 * tp / (2 * tp + fp + fn)
FAR = fp/(fp + tn)
FRR = fn/(fn + tp)
HTER_val = (FAR + FRR)/2
EER = (fp+fn)/(tn+fp+fn+tp)
val_bacc = balanced_accuracy_score(test_cat_labels_cpu, test_predicted_cat_labels_cpu)


print('Testing Results')
print(30*'-')
print('Acc:', acc_score, '\nSen:', sensitivity_val, '\nSpec:', specificity_val, '\nYI:', Y_I_val, '\nF1:', f1score_val, '\nPrec:', prec_score, '\nRecall:', recall, '\nHTER:', HTER_val, '\nEER:', EER, '\nBACC:', val_bacc)